In [ ]:
import sys
import ipywidgets as ipyw
import datetime
import requests
import json
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
from IPython.display import display, clear_output, HTML, Javascript

cols = ['1','2','3','4','5','6','7','8']

def getSelectedCols():
    arr = []
    for item in buttons:
        if item.value == True:
            arr.append(item.description)
    return arr

def addDatetimeCol(results):
    dtArr = []
    for item in results:
        dt = item["published_at"][:-1]
        dtArr.append(np.datetime64(dt))
    df = pd.DataFrame(results)
    df.insert(0, "datetime", dtArr)
    return df

def getData(b):
    headers = {'Content-type': 'application/json'}
    selectedCols = getSelectedCols()
    
    if len(selectedCols) > 0:
        proj = dict.fromkeys(selectedCols, True)
        proj["published_at"] = True
        data = {"method":"query", 
                "query":{"published_at":{"$gte":dateFrom.value.strftime('%Y-%m-%d') + "T00:00:00", 
                                "$lt":dateTo.value.strftime('%Y-%m-%d') + "T23:23:59"}}, 
                "projection":proj}
        print(data)
        
        # Get Data
        res = requests.post('http://149.165.157.11:4793/airqualitysensordb/query', data=json.dumps(data), headers=headers)
        results = json.loads(res.content)
        
        print("Size: " + str(len(results)))

        df = addDatetimeCol(results)
        
        # Setup plot figure
        numCol = len(selectedCols)
        fig = make_subplots(rows=numCol, cols=1)
        
        # Draw plots
        for i in range(numCol):
            fig.append_trace(go.Scatter(x = df['datetime'], y = df[selectedCols[i]], name=selectedCols[i], mode='markers'), row=i+1, col=1)

        with displayArea:
            clear_output()
            
            fig.update_layout(height=200*numCol, width=1000, title_text="Stacked Subplots")
            fig.show()


dateFrom = ipyw.DatePicker(value=datetime.date(2021, 2, 16), description='Start Date', disabled=False, layout=ipyw.Layout(width='250px'))
dateTo = ipyw.DatePicker(value=datetime.date(2021, 2, 17), description='End Date', disabled=False, layout=ipyw.Layout(width='250px'))

inputParams = ipyw.VBox([dateFrom, dateTo], layout=ipyw.Layout(margin="25px 0px 0px 0px"))

buttons = []
for item in cols:
    buttons.append(ipyw.Checkbox(value=False,indent=True, description=item))

checkButtons = ipyw.VBox(buttons)

plotButton = ipyw.Button(description = "Get & Plot", button_style = "success",)
plotButton.on_click(getData)
plotBox = ipyw.Box([plotButton], layout = ipyw.Layout(display='flex', flex_flow='column', align_self='center', width='50%'))

paramBox = ipyw.VBox([inputParams, checkButtons, plotBox])
displayArea = ipyw.Output()
displayBox = ipyw.Box([displayArea])
display(ipyw.Box([paramBox, displayBox]))
